# database upload

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import psycopg2
import datetime as dt
from delta import *
import boto3
import pprint
import yaml
import time
import json
import sys
import ast
import os

pp = pprint.PrettyPrinter(indent = 1)
print("imported modules.")

imported modules.


# creds & config

In [2]:
creds_path = os.path.join("/opt", "workspace", "redditStreaming", "src", "main", "python", "reddit", "creds.json")
config_path = os.path.join("/opt", "workspace", "redditStreaming", "src", "main", "python", "reddit", "config.yaml")

try:
    with open(creds_path, "r") as f:
        creds = json.load(f)
        print("read creds.json.")
        f.close()

    with open(config_path, "r") as g:
        config = yaml.safe_load(g)
        print("read config.yaml.")
        g.close()

    print("read creds & config.")

except:
    creds_path = "/home/steven/Documents/reddit-streaming/redditStreaming/src/main/python/reddit/creds.json"
    with open(creds_path, "r") as f:
        creds = json.load(f)
        print("read creds.json.")
        f.close()

print("read creds successfully.")

read creds.json.
read config.yaml.
read creds & config.
read creds successfully.


# spark session

In [3]:
spark_host = "spark-master"
# spark_host = "spark-master"
aws_client = creds["aws_client"]
aws_secret = creds["aws_secret"]
index = 0
subreddit = "technology"

# initialize spark session
try:
    spark = SparkSession.builder.appName("reddit_{}".format(subreddit)) \
                .master("spark://{}:7077".format(spark_host)) \
                .config("spark.scheduler.mode", "FAIR") \
                .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
                .config("spark.executor.memory", "4096m") \
                .config("spark.executor.cores", "4") \
                .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
                .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
                .config("spark.eventLog.enabled", "true") \
                .config("spark.eventLog.dir", "file:///opt/workspace/events/{}/".format(subreddit)) \
                .config("spark.sql.debug.maxToStringFields", 1000) \
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-core_2.12:1.2.1,org.postgresql:postgresql:42.5.0") \
                .config("spark.hadoop.fs.s3a.access.key", aws_client) \
                .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                .config('spark.hadoop.fs.s3a.buffer.dir', '/opt/workspace/tmp/blocks') \
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
                .enableHiveSupport() \
                .getOrCreate()

    sc = spark.sparkContext
    # .config('spark.hadoop.fs.s3a.fast.upload.buffer', 'bytebuffer') \

    sc.setLogLevel('WARN')
    sc.setLocalProperty("spark.scheduler.pool", "pool{}".format(str(index)))
    # sc._jsc.hadoopConfiguration().set("fs.s3a.awsAccessKeyId", aws_client)
    # sc._jsc.hadoopConfiguration().set("fs.s3a.awsSecretAccessKey", aws_secret)
    # sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.us-east-2.amazonaws.com")
    print("created spark successfully")

except Exception as e:
    print(e)

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d721c81-1caf-4f20-af1b-c2054e4fce23;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in centr

23/02/22 01:49:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/22 01:49:48 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
created spark successfully


# spark

In [4]:
spark

# read df

In [6]:
bucket = config["bucket"]
folder = "aws_clean"
read_path = "s3a://" + bucket + "/" + folder
print(read_path)
df = spark.read.format("delta").option("header", True).load(read_path)

s3a://reddit-streaming-stevenhurwitt-new/aws_clean
23/02/22 01:50:19 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


# show df

In [12]:
pandas_df = df.toPandas()

In [13]:
pandas_df

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,date,year,month,day
0,NaT,aws,I have this PHP (Laravel) script to get all th...,t2_6j0lp,False,None,0,False,Weird behavior getting a list of S3 object keys,r/aws,...,https://www.reddit.com/r/aws/comments/10t6cw6/...,228560,2023-02-04 05:48:13.401088,0,None,False,2023-02-04,2023,2,4
1,NaT,aws,I'm not exactly on the up and up on some of th...,t2_jsvj0raw,False,None,0,False,Pattern for ingesting deltas and merging into ...,r/aws,...,https://www.reddit.com/r/aws/comments/10pmcs3/...,228234,2023-01-31 03:00:16.421888,0,None,False,2023-01-31,2023,1,31
2,NaT,aws,"Hey there,\n\nIs anyone using the AWS Transfer...",t2_4b72e,False,None,0,False,AWS Transfer Family price question,r/aws,...,https://www.reddit.com/r/aws/comments/10p0t6t/...,228218,2023-01-30 12:32:19.943424,0,None,False,2023-01-30,2023,1,30
3,NaT,aws,"I can't imagine the answer is ""it does scale"",...",t2_52j9a,False,None,0,False,How does (does?) API Keys / Usage Plans / WAF ...,r/aws,...,https://www.reddit.com/r/aws/comments/10p3zl9/...,228191,2023-01-30 14:28:39.265280,0,None,False,2023-01-30,2023,1,30
4,NaT,aws,"Hi,\n\nI’m looking at using SES for email list...",t2_3sde0,False,None,0,False,SES Number of Contact Lists Per Account,r/aws,...,https://www.reddit.com/r/aws/comments/10opryt/...,228222,2023-01-30 02:08:13.197312,0,None,False,2023-01-30,2023,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,NaT,aws,I have data in a DDB. I want to make visualiza...,t2_uf837,False,None,0,False,Optimal way to visualize your data from DDB,r/aws,...,https://www.reddit.com/r/aws/comments/10s3qcg/...,228478,2023-02-02 22:58:15.340032,0,None,False,2023-02-02,2023,2,2
70,NaT,aws,"So, I'm trying to learn how to use ECS to port...",t2_6jr6y,False,None,0,False,Cloudformation: Is it just really bad for ever...,r/aws,...,https://www.reddit.com/r/aws/comments/10pjtk0/...,228228,2023-01-31 01:03:57.100032,0,None,False,2023-01-31,2023,1,31
71,NaT,aws,I created a [public hosted zone](https://i.img...,t2_e8so8,False,None,0,False,Why can't I see my domain's nameservers hosted...,r/aws,...,https://www.reddit.com/r/aws/comments/10t7vbt/...,228564,2023-02-04 07:19:56.327936,0,None,False,2023-02-04,2023,2,4
72,NaT,aws,I have a project to add ocr and search functio...,t2_ia3tn,False,None,0,False,search document text,r/aws,...,https://www.reddit.com/r/aws/comments/10t349w/...,228554,2023-02-04 02:53:44.418304,0,None,False,2023-02-04,2023,2,4


# write df

In [8]:
write_path = "s3a://" + bucket + "final_aws_clean/"
print(write_path)
# df.write.format("delta").option("header", True).save(write_path)

s3a://reddit-streaming-stevenhurwitt-newfinal_aws_clean/


In [ ]:
# spark.stop()

## read clean df

In [14]:
def write_spark_jdbc(subreddit):

    creds_path = os.path.join("/opt", "workspace", "redditStreaming", "creds.json")

    try:
        with open(creds_path, "r") as f:
            creds = json.load(f)
            print("read creds.json.")
            f.close()

    except:
        creds_path = "/home/steven/Documents/reddit-streaming/redditStreaming/creds.json"
        with open(creds_path, "r") as f:
            creds = json.load(f)
            print("read creds.json.")
            f.close()

    secretmanager_client = boto3.client("secretsmanager", 
                                    region_name = "us-east-2", 
                                    aws_access_key_id = creds["aws_client"], 
                                    aws_secret_access_key = creds["aws_secret"])
    
    df = spark.read.format("delta").option("header", True).load("s3a://reddit-streaming-stevenhurwitt/" + subreddit + "_clean")

    db_creds = ast.literal_eval(secretmanager_client.get_secret_value(SecretId="dev/reddit/postgres")["SecretString"])
    connect_str = "jdbc:postgresql://{}:5432/reddit".format(db_creds["host"])

    try:
        df.write.format("jdbc") \
            .mode("overwrite") \
            .option("url", connect_str) \
            .option("dbtable", "reddit.{}".format(subreddit)) \
            .option("user", db_creds["username"]) \
            .option("password", db_creds["password"]) \
            .option("driver", "org.postgresql.Driver") \
            .save()

        print("wrote df to postgresql table.")

    except Exception as e:
        print(e)

In [15]:
config["subreddit"]

['technology',
 'ProgrammerHumor',
 'news',
 'worldnews',
 'BikiniBottomTwitter',
 'BlackPeopleTwitter',
 'WhitePeopleTwitter',
 'aws']

In [39]:
for s in config["subreddit"]:
    print("subreddit: {}".format(s))
    write_spark_jdbc(s)

subreddit: technology
read creds.json.
23/02/17 05:16:50 WARN DeltaLog: Failed to parse s3a://reddit-streaming-stevenhurwitt/technology_clean/_delta_log/_last_checkpoint. This may happen if there was an error during read operation, or a file appears to be partial. Sleeping and trying again.
java.nio.file.AccessDeniedException: s3a://reddit-streaming-stevenhurwitt/technology_clean/_delta_log/_last_checkpoint: getFileStatus on s3a://reddit-streaming-stevenhurwitt/technology_clean/_delta_log/_last_checkpoint: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 7EGNZAC6M0AT0Q40; S3 Extended Request ID: JQKBA2rHdpQdTfMrrXbsqDK3bNeYxjIqUnU1jksIHrVLKfD5gZGzigH15bdxDcSHgaLy7Ckcj+Y=; Proxy: null), S3 Extended Request ID: JQKBA2rHdpQdTfMrrXbsqDK3bNeYxjIqUnU1jksIHrVLKfD5gZGzigH15bdxDcSHgaLy7Ckcj+Y=:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:249)
	at org.apache.hadoop.fs

Py4JJavaError: An error occurred while calling o98.load.
: java.util.concurrent.ExecutionException: java.nio.file.AccessDeniedException: s3a://reddit-streaming-stevenhurwitt/technology_clean/_delta_log: getFileStatus on s3a://reddit-streaming-stevenhurwitt/technology_clean/_delta_log: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: BMWFQHRCMTY0VW8V; S3 Extended Request ID: rTuwE5yq2JVxR0Edwhnkf70yX7qtwl4LvQX9YrBx1Sd6zqU7Hf8EoMrlmbBMv78zibahnVO4ZhI=; Proxy: null), S3 Extended Request ID: rTuwE5yq2JVxR0Edwhnkf70yX7qtwl4LvQX9YrBx1Sd6zqU7Hf8EoMrlmbBMv78zibahnVO4ZhI=:403 Forbidden
	at com.google.common.util.concurrent.AbstractFuture$Sync.getValue(AbstractFuture.java:306)
	at com.google.common.util.concurrent.AbstractFuture$Sync.get(AbstractFuture.java:293)
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:116)
	at com.google.common.util.concurrent.Uninterruptibles.getUninter

23/02/17 05:17:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:17:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/02/17 05:17:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:17:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:18:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:18:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/02/17 05:18:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:18:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:19:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:19:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


...

23/02/17 05:20:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:20:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:21:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:21:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/02/17 05:21:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:21:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:22:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:22:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/02/17 05:22:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:22:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:23:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:23:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/02/17 05:23:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:23:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:24:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:24:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


23/02/17 05:24:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:24:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/02/17 05:25:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


## write to postgres table

### create schema

In [ ]:
conn = psycopg2.connect(host = config["postgres_host"], user = config["postgres_user"], password = config["postgres_password"], database="postgres")

OperationalError: could not connect to server: Connection refused
	Is the server running on host "xanaxprincess.asuscomm.com" (174.170.113.213) and accepting
	TCP/IP connections on port 5432?


In [ ]:
connect_str = "jdbc:postgresql://{}:5432/postgres".format(config["postgres_host"])

try:
    df.write.format("jdbc") \
        .mode("overwrite") \
        .option("url", connect_str) \
        .option("dbtable", "public.{}".format(subreddit)) \
        .option("user", config["postgres_user"]) \
        .option("password", config["postgres_password"]) \
        .option("driver", "org.postgresql.Driver") \
        .save()

    print("wrote df to postgresql table.")

except Exception as e:
    print(e)

name 'df' is not defined


23/02/17 05:09:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


## stop spark

In [24]:
try:
    spark.stop()

except Exception as e:
    print(e)

SparkSession does not exist in the JVM
